In [1]:
pip install transformers datasets sentencepiece sacrebleu evaluate 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 69.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import Dataset

# # Load the Parquet file
df = pd.read_csv('training_dataset/documentary_commentary_tatoeba_ccmatrix_europarl_opensubtitle.csv')
df = df.drop_duplicates()
df = df.dropna()
df = df.reset_index()
df = df.drop(columns='index')
def capitalize_first_letter(text):
    return text.capitalize()

df['english'] = df['english'].apply(capitalize_first_letter)
df['french'] = df['french'].apply(capitalize_first_letter)

# dataset = Dataset.from_pandas(df)
# dataset


In [4]:
df

,english,french,tag
0,I wonder some times how some people get publis...,Je me demande parfois comment certaines person...,documentary
1,You do not need to be an expert to know this p...,Vous n'avez pas besoin d'être un expert pour s...,documentary
2,"Some things are better just left, benidorm is ...",Certaines choses sont mieux il suffit de laiss...,documentary
3,With a good series you are left wanting just a...,Avec une bonne série vous êtes laissé vouloir ...,documentary
4,"Think lucifer has done its day, but preacher h...","Pensez que lucifer a fait son jour, mais le pr...",documentary
...,...,...,...
2417586,It does not require a constitutional amendment...,"Elle n'exige pas d'amendement constitutionnel,...",europarl
2417587,"Mr president, i want to add my congratulations...","Monsieur le président, je souhaiterais, moi au...",europarl
2417588,Are we mad to be here at this time of night de...,Sommes-nous fous de rester dans cette assemblé...,europarl
2417589,"Really, it is the people who are not here who ...","En réalité, ce sont les absents qui doivent co...",europarl


In [5]:
# import pandas as pd
# from datasets import Dataset

# # # Load the Parquet file
# df1 = pd.read_csv('training_dataset/discussion_news_2019_cleaned_0to6.csv')
# df1 = df1.drop_duplicates()
# df1 = df1.dropna()
# df1 = df1.reset_index()
# df1 = df1.drop(columns='index')
# def capitalize_first_letter(text):
#     return text.capitalize()

# df1['english'] = df1['english'].apply(capitalize_first_letter)
# df1['french'] = df1['french'].apply(capitalize_first_letter)



In [6]:
# df1

In [7]:
# result = pd.concat([df1, df], axis=0)
# result

In [8]:
dataset = Dataset.from_pandas(df)
dataset


Dataset({
    features: ['english', 'french', 'tag'],
    num_rows: 2417591
})

In [9]:
from transformers import MarianTokenizer, MarianMTModel

model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name) 

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
def preprocess(batch):
    target_texts = batch['english']  # list of English texts
    input_texts = batch['french']    # list of French texts

    # Tokenize inputs and targets in batch
    model_inputs = tokenizer(
        input_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    labels = tokenizer(
        target_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=["english", "french"], batched=True)


Map:   0%|          | 0/2417591 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

In [11]:
tokenized_dataset 

Dataset({
    features: ['tag', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2417591
})

In [12]:
from datasets import ClassLabel

# Get the unique speaker_ids as strings
unique_tag_ids = list(map(str, sorted(set(tokenized_dataset['tag']))))

# Create a ClassLabel object using these unique string labels
filtered_dataset = tokenized_dataset.cast_column('tag', ClassLabel(names=unique_tag_ids))
filtered_dataset

Casting the dataset:   0%|          | 0/2417591 [00:00<?, ? examples/s]

Dataset({
    features: ['tag', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2417591
})

In [13]:
filtered_dataset

Dataset({
    features: ['tag', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2417591
})

In [14]:
# tokenized_dataset[0]
train_val_split = filtered_dataset.train_test_split(test_size=0.1, shuffle=True,stratify_by_column='tag', seed=42)
train_dataset = train_val_split ['train'].remove_columns("tag")
test_dataset = train_val_split ['test'].remove_columns("tag")

In [15]:
import numpy as np
import sacrebleu
from transformers import EvalPrediction
def compute_bleu(eval_pred):
    preds, labels = eval_pred

    # Decode generated sequences
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Decode labels (ignore padding / -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Strip and wrap refs for sacrebleu
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [[l.strip()] for l in decoded_labels]

    # Compute BLEU
    bleu = sacrebleu.corpus_bleu(decoded_preds, decoded_labels)
    
    return {"bleu": bleu.score}

In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./marian-fr-en-finetuned",
    per_device_train_batch_size=128, # lower is better for generalization and higher variance, higher is the opposite and could overfit
    per_device_eval_batch_size=128, # do 128 after
    warmup_steps = 1000, # prevent overfitting
    gradient_accumulation_steps=2, #prevent overfitting 
    learning_rate=1e-5, #find the complexity of the model to find the right learning range
    num_train_epochs=8, 
    weight_decay=0.01, # to 1
    logging_steps=10000,
    predict_with_generate=True, #True with compute_metrics
    generation_max_length=128,  #True with compute_metrics
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy='epoch',
    save_total_limit=1,
    fp16=True,
    metric_for_best_model="bleu", 
    report_to="none",
    seed=42,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_bleu
)
trainer.train()


/tmp/ipykernel_376/692418019.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.230847,64.254293
2,0.375100,0.211579,64.763821
3,0.233000,0.203513,64.763821
4,0.218200,0.198972,70.456776
5,0.211000,0.196339,72.337274
6,0.206900,0.194621,72.337274
7,0.206900,0.193603,75.242177
8,0.204100,0.193282,75.242177


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=68000, training_loss=0.23686360617244945, metrics={'train_runtime': 84796.1259, 'train_samples_per_second': 205.276, 'train_steps_per_second': 0.802, 'total_flos': 5.900569060772413e+17, 'train_loss': 0.23686360617244945, 'epoch': 8.0})

In [17]:
output_dir = "/workspace/marianmt_fine_tuned11"

trainer.save_model(output_dir) 
tokenizer.save_pretrained(output_dir) 

('/workspace/marianmt_fine_tuned11/tokenizer_config.json',
 '/workspace/marianmt_fine_tuned11/special_tokens_map.json',
 '/workspace/marianmt_fine_tuned11/vocab.json',
 '/workspace/marianmt_fine_tuned11/source.spm',
 '/workspace/marianmt_fine_tuned11/target.spm',
 '/workspace/marianmt_fine_tuned11/added_tokens.json')